In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [15]:
no_pages = 6

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [16]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [17]:
df.shape

(100, 5)

In [19]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
df['Rating'].head()

0    4.5
1    4.6
2    4.6
3    4.4
4    4.4
Name: Rating, dtype: object

In [18]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"27,035",₹399.00
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"16,704",₹299.00
2,The Psychology of Money,Morgan Housel,4.6 out of 5 stars,"11,681",₹295.00
3,My First Book of Pencil Control : Practice Pat...,Wonder House Books,4.4 out of 5 stars,"6,688",₹89.00
4,Word Power Made Easy,Norman Lewis,4.4 out of 5 stars,"22,773",₹90.00
...,...,...,...,...,...
56,Peppa Pig: Little Library,Peppa Pig,4.3 out of 5 stars,"14,124",₹198.00
57,Capital Letters ABC: Write and practice Capita...,Wonder House Books,4.3 out of 5 stars,"1,231",₹42.00
58,Amazon Brand - Solimo Board Books for Kids (Se...,Solimo,4.5 out of 5 stars,"4,445",₹480.00
59,Mahatma Gandhi Autobiography: The Story Of My ...,Mahatma Gandhi,4.5 out of 5 stars,"3,330",₹99.00


In [26]:
df['Rating'] = pd.to_numeric(df['Rating'])

In [27]:
df["Price"] = df["Price"].str.replace('₹', '')

In [28]:
df["Price"] = df["Price"].str.replace(',', '')

In [29]:
df["Price"].head()

0    399
1    299
2    295
3     89
4     90
Name: Price, dtype: object

In [30]:
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])

In [31]:
df['Price'] = df['Price'].astype(int)

In [32]:
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')

In [33]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')

In [34]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,27035,399
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,16704,299
2,The Psychology of Money,Morgan Housel,4.6,11681,295
3,My First Book of Pencil Control : Practice Pat...,Wonder House Books,4.4,6688,89
4,Word Power Made Easy,Norman Lewis,4.4,22773,90


In [35]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int32
dtype: object